## Evaluation Dataset Preparation

In [1]:
import pandas as pd
import numpy as np
import random
import ast
import nlpaug.augmenter.word as naw
from math import sqrt
from sklearn.model_selection import train_test_split

2023-03-04 14:13:32.757170: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


2023-03-04 14:13:34.143525: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-03-04 14:13:34.143609: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory
2023-03-04 14:13:34.143618: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nvidia GPU with TensorRT, please make sure the missing libraries mentioned above are installed properly.


In [2]:
df = pd.read_csv('../data/lyrics.csv')

In [3]:
df.head()

,lyrics,genre,id
0,"['So long', 'My sweet love', 'I miss you', ""Mo...",pop,0
1,"[""It won't be long before I'm leaving here"", ""...",pop,1
2,"['Are we crazy?', 'Living our lives through a ...",pop,2
3,"['When did it go wrong, I will never know', 'I...",pop,3
4,"[""I've waited all this time"", 'Counting minute...",pop,4


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36000 entries, 0 to 35999
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   lyrics  36000 non-null  object
 1   genre   36000 non-null  object
 2   id      36000 non-null  int64 
dtypes: int64(1), object(2)
memory usage: 843.9+ KB


In [5]:
df['lyrics'] = df['lyrics'].apply(ast.literal_eval)

In [6]:
df0 = df.copy()

In [7]:
df = df.groupby("genre").sample(n=50, random_state=42).reset_index()

In [8]:
eg_lyrics = random.choice(df['lyrics'])
eg_lyrics

['Warning, warning, warning, the drama queen is in the building',
 'The drama queen is in the building',
 "I'm not the one, I'm not the one",
 'Everybody wanna talk about kim',
 "But ain't nothing soft about kim",
 'Everything you walk about kim',
 "Through all that I've been",
 'Picture me being broke and I can auction my pen, please',
 "I ain't never worry about hens, they worry about me",
 "But I ain't worry about them",
 "I'd rather see them coughing up flem",
 "Off with they heads, guillotine kim so it's off with they limbs",
 'Why pop cork with my men',
 'When I was up north in the pen, my fans wrote me',
 'They said queen keep doing ya thing right songs so we can quote thee',
 "The rest of ya'll we",
 "See our downfall, so I don't come around y'all",
 "Now I'm beyond rich, the spittin magician poof be gone bitch",
 "Bitch couldn't walk a mile in my louboutins",
 "Me against you that's asalan",
 'Qb bout the bomb, taliban',
 'Coming through the north trench, caliban',
 "Bitch you

In [9]:
aug = naw.ContextualWordEmbsAug(model_path='roberta-base', action="substitute")

In [10]:
def misalign_lyrics(lyrics, repetitions, repeat_verse):
    verses2repeat = random.sample(range(0, len(lyrics)), repetitions)
    misaligned_lyrics = []
    for idx, verse in enumerate(lyrics):
        if idx in verses2repeat:
            misaligned_lyrics.extend([verse] * repeat_verse)
        else:
            misaligned_lyrics.append(verse)
    return misaligned_lyrics

In [11]:
evaluation_ids = list(df['id'])

all_lyrics = df0.drop(index=evaluation_ids)['lyrics']

In [12]:
eg_verse = random.choice(eg_lyrics)
print('Original verse:')
print(eg_verse)

print('Augmented:')
aug.augment(eg_verse)

Original verse:
When I was up north in the pen, my fans wrote me
Augmented:


['When Mark was up and in the pen, Jets fans missed me']

In [13]:
eg_lyrics = random.choice(df['lyrics'])

eg_aug_lyrics = aug.augment(eg_lyrics)

eg_aug_misl_lyrics = misalign_lyrics(eg_aug_lyrics, 
                                     repetitions=round(sqrt(len(eg_lyrics))), 
                                     repeat_verse=2)

eg_fake_lyrics = random.choice(all_lyrics)

print('ORIGINAL lyrics:')
print(eg_lyrics)
print()
print('AUGMENTED lyrics:')
print('Aligned:')
print(eg_aug_lyrics)
print()
print('Misaligned:')
print(eg_aug_misl_lyrics)
print()
print('RANDOM lyrics:')
print(eg_fake_lyrics)

ORIGINAL lyrics:
['Mirror, mirror on the wall', 'Tell me, mirror, what is wrong?', 'Can it be my De La clothes', 'Or is it just my De La song?', "What I do ain't make-beleive", 'People say I sit and try', 'But whan it comes to being De La', "It's just me myself and I", "It's just me myself and I", "It's just me myself and I", "It's just me myself and I", 'Now you tease my Plug One style', 'And my Plug One spectacles', 'You say Plug One and Two are hippies', "No, we're not, that's pure Plug bull", "Always pushing that we've formed an image", "There's no need to lie", 'When it comes to being Plug One', "It's just me myself and I", "It's just me myself and I", "It's just me myself and I", "It's just me myself and I", "Proud, I'm proud of what I am", 'Poems I speak are Plug Two type', 'Please oh please let Plug Two be', 'Himself, not what you read or write', 'Right is wrong when hype is written', 'On the Soul, De La that is,', 'Style is surely our own thing', 'Not the false disguise of sho

In [14]:
aug_lyrics = df['lyrics'].apply(lambda x: aug.augment(x))


misl_aug_lyrics = [misalign_lyrics(aug_lyr, repetitions=round(sqrt(
    len(aug_lyr))), repeat_verse=random.choice([2,3])) for aug_lyr in aug_lyrics]

                                   
df['similar_lyrics'] = misl_aug_lyrics
df['random_lyrics'] = random.sample(list(all_lyrics), len(df['lyrics']))

In [15]:
df.head()

,index,lyrics,genre,id,similar_lyrics,random_lyrics
0,1935,"[Well, I've been waitin' ever since eight, Gue...",pop,1935,"[Well, never been happily' ever after …, Guess...","[I've seen a lot of things in my life, A lot o..."
1,6494,"[This kind of love, Is more than a lifeline, F...",pop,6494,"[This kind about one, Is more new a reception,...","[Tell 'em, go home 'cause I'ma go hard, I'ma g..."
2,1720,"[I could tell by the look in her eyes, Maybe I...",pop,1720,"[I would tell by the look must too eyes, Maybe...","[Let it rain, let it snow, Let it rain, let it..."
3,9120,"[Don't Tell me you love me when I don't, You k...",pop,9120,"[Don't Tell Anyone you trust me else I A, You ...","[I feel weak, we've been here before, 'Cause I..."
4,360,"[Who's the man? who's the boss?, who's the nig...",pop,360,"[Who's i they? who's I boss?, Who's i they? wh...","[Now this shit right here, Goes out all my cat..."


In [16]:
df.to_csv('../data/evaluation/lyrics_eval.csv', index=False)